In [2]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy

In [3]:
ufo_raw = pd.read_csv("data/ufo_scrubbed.csv")


C:\Users\hfatt\anaconda3\envs\Pythondata\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
ufo_raw.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [5]:
ufo_raw["datetime"] = pd.to_datetime(ufo_raw["datetime"], format="%m/%d/%Y %H:%M", errors='coerce')

In [6]:
ufo_raw[ufo_raw["datetime"].isna()]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
388,NaT,rome,ny,us,oval,120,a min or two,I was walking from the garage to the house&#44...,2/1/2007,43.2127778,-75.456111
693,NaT,chulucanas-piura la vieja (peru),NaN,NaN,other,6312000,2 years,go to: http://www.24horas.com.pe/data/videos/...,3/4/2003,-5.129547,-80.120569
962,NaT,novi,mi,us,triangle,300,5 minutes,V shaped and 8 big and very brite lights&#44mo...,10/30/2012,42.4805556,-83.475556
1067,NaT,salatiga (indonesia),NaN,NaN,disk,22,22 seconds,UFO in Salatiga&#44Indonesia,10/31/2003,-7.33683,110.498817
1221,NaT,cincinnati,oh,us,fireball,300,3-5 minutes,A bright orange light split into four&#44 did ...,10/14/2013,39.1619444,-84.456944
...,...,...,...,...,...,...,...,...,...,...,...
79136,NaT,boonville,nc,us,cigar,10800,3+ hours,Unusual activity in our small community&#44 cr...,9/9/2013,36.2325,-80.708333
79137,NaT,redondo beach,ca,us,triangle,300,5 minutes,Triangulat UFO seen over Redondo Beach&#44 Cal...,12/16/2005,33.8492,-118.387500
79740,NaT,portsmouth,oh,us,changing,900,15 minutes,A bright red light hoovering in the sky.,9/13/2002,38.7317,-82.997778
79759,NaT,montrose,co,us,unknown,1200,20 minutes,Stationary object with three pulsating lights ...,9/9/2004,38.4783,-107.875556


In [14]:
us_census = pd.read_csv("data/US_census.csv")
us_census.head()

,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,Resident Population Density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative
0,Alabama,State,1910,"2,138,093",16.9,42.2,25.0,10.0,1.0,"213,809"
1,Alaska,State,1910,"64,356",1.2,0.1,52.0,NaN,NaN,NaN
2,Arizona,State,1910,"204,354",66.2,1.8,49.0,NaN,NaN,NaN
3,Arkansas,State,1910,"1,574,449",20.0,30.3,30.0,7.0,0.0,"224,921"
4,California,State,1910,"2,377,549",60.1,15.3,38.0,11.0,3.0,"216,051"


In [15]:
us_census = us_census[us_census["Year"] >= 1950]
us_census.head()

,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,Resident Population Density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative
228,Alabama,State,1950,"3,061,743",8.1,60.5,24.0,9.0,0.0,"340,194"
229,Alaska,State,1950,"128,643",77.4,0.2,52.0,NaN,NaN,NaN
230,Arizona,State,1950,"749,587",50.1,6.6,47.0,2.0,0.0,"374,794"
231,Arkansas,State,1950,"1,909,511",-2.0,36.7,34.0,6.0,-1.0,"318,252"
232,California,State,1950,"10,586,223",53.3,68,22.0,30.0,7.0,"352,874"


In [20]:
az_census = us_census[us_census["Name"] == "Arizona"]
ca_census = us_census[us_census["Name"] == "Calfornia"]
fl_census = us_census[us_census["Name"] == "Florida"]
il_census = us_census[us_census["Name"] == "Illinois"]
mi_census = us_census[us_census["Name"] == "Michigan"]
ny_census = us_census[us_census["Name"] == "New York"]
oh_census = us_census[us_census["Name"] == "Ohio"]
or_census = us_census[us_census["Name"] == "Oregon"]
pa_census = us_census[us_census["Name"] == "Pennsylvania"]
tx_census = us_census[us_census["Name"] == "Texas"]
wa_census = us_census[us_census["Name"] == "Washington"]
az_census

,Name,Geography Type,Year,Resident Population,Percent Change in Resident Population,Resident Population Density,Resident Population Density Rank,Number of Representatives,Change in Number of Representatives,Average Apportionment Population Per Representative
230,Arizona,State,1950,"749,587",50.1,6.6,47.0,2.0,0.0,"374,794"
287,Arizona,State,1960,"1,302,161",73.7,11.5,43.0,3.0,1.0,"434,054"
344,Arizona,State,1970,"1,770,900",36.0,15.6,43.0,4.0,1.0,"446,905"
401,Arizona,State,1980,"2,718,215",53.5,23.9,42.0,5.0,1.0,"543,573"
458,Arizona,State,1990,"3,665,228",34.8,32.3,39.0,6.0,1.0,"612,998"
515,Arizona,State,2000,"5,130,632",40.0,45.2,38.0,8.0,2.0,"642,585"
572,Arizona,State,2010,"6,392,017",24.6,56.3,35.0,9.0,1.0,"712,522"
629,Arizona,State,2020,"7,151,502",11.9,62.9,35.0,9.0,0.0,"795,436"


In [21]:
focus_states = ["Arizona", "Calfornia", "Florida", "Illinois", "Michigan", "New York", "Ohio", "Oregon", 
          "Pennsylvania", "Texas" "Washington"]

